In [ ]:
# instalujemy potrzebne biblioteki / możliwe że trzeba będzie zrobić restart kernela
!pip install --quiet faker

In [ ]:
import random
import sqlite3

import pandas as pd
from faker import Faker

In [ ]:
fake = Faker()

# zanim zaczniemy stworzymy sobie losowe dane
employees = []
for i in range(1000):
    employees.append({
        "id": i + 1,
        'name': fake.first_name(),
        'surname': fake.last_name(),
        'age': random.randint(20, 80) if random.random() > 0.1 else None,
        'salary': random.randint(3000, 10000),
        'department': random.choice(['IT', 'HR', 'Finance', 'Marketing'])
    })

# tworzymy DataFrame
df = pd.DataFrame(employees)

# zapisujemy do bazy danych
with sqlite3.connect('learning_select.db') as conn:
    df.to_sql('employees', conn, if_exists='replace', index=False)

In [ ]:
# contracts
contracts = []
# 300 pracowników pracuje bez umowy ;-)
for i in range(700):
    contracts.append({
        'employee_id': i + 1,
        'type': random.choice(['B2B', 'UoP']),
        'start': fake.date_between(start_date='-5y', end_date='today'),
        'end': fake.date_between(start_date='today', end_date='+5y'),
    })

# 300 innych już nie pracuje
for i in range(700, 1000):
    contracts.append({
        'employee_id': 1000 + i + 1,
        'type': random.choice(['B2B', 'UoP']),
        'start': fake.date_between(start_date='-5y', end_date='-1y'),
        'end': fake.date_between(start_date='-1y', end_date='today'),
    })

# tworzymy DataFrame
df = pd.DataFrame(contracts)

# zapisujemy do bazy danych
with sqlite3.connect('learning_select.db') as conn:
    df.to_sql('contracts', conn, if_exists='replace', index=False)

# Baza danych - SQL - język baz danych - jak prosić o dane?

Podstawową instrukcją w języku SQL do proszenia o dane jest `SELECT`

## Jak wczytać wszystkie wiersze / rekordy?

In [ ]:
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('SELECT * FROM employees', conn)

df

## Czy w zapytaniach SQL słowa wbudowane trzeba pisać dużymi literami?

In [ ]:
# nie trzeba słów kluczowych SQL pisac wielkimi literami
# ALE trzymanie się tej konwencji jest dobre dla czytelności
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('select * from employees', conn)

df

## Jak wczytać tylko wybrane kolumny?

In [ ]:
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('SELECT name, surname, salary FROM employees', conn)

df

## Jak wczytać wybrane kolumny z bazy danych z aliasami?

In [ ]:
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql(
        'SELECT name AS imie, surname AS nazwisko, salary AS pensja FROM employees',
        conn,
    )

df

## Jak filtrować dane w bazie danych?

In [ ]:
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('SELECT * FROM employees WHERE age > 50', conn)

df

In [ ]:
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql("SELECT * FROM employees WHERE department = 'IT'", conn)

df

In [ ]:
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('SELECT * FROM employees WHERE department = "HR"', conn)

df

## Jak budować bardziej złożone filtry / kryteria?

In [ ]:
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('''
        SELECT
            *
        FROM
            employees
        WHERE department = "HR" AND age > 50
    ''', conn)

df

In [ ]:
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('''
        SELECT
            *
        FROM
            employees
        WHERE
            (department = "HR" AND age > 50) OR
            (department = "IT" AND age < 30)
    ''', conn)

df

## Jak używać `IN` w kryteriach?

In [ ]:
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('''
        SELECT
            *
        FROM
            employees
        WHERE
            department IN ("HR", "IT")
    ''', conn)

df["department"].value_counts()

## Jak znaleźć rekordy gdzie brakuje wartości?

In [ ]:
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('''
        SELECT
            *
        FROM
            employees
        WHERE
            age IS NULL
    ''', conn)

df

In [ ]:
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('''
        SELECT
            *
        FROM
            employees
        WHERE
            age IS NOT NULL
    ''', conn)

df

## Jak przeszukiwać kolumny, które zawierają łańcuchy znaków?

In [ ]:
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('''
        SELECT
            name, surname, salary
        FROM
            employees
        WHERE
            name LIKE "A%"
    ''', conn)

df

In [ ]:
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('''
        SELECT
            name, surname, salary
        FROM
            employees
        WHERE
            name LIKE "al%"
    ''', conn)

df

## Jak filtrować po datach?

In [ ]:
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('''
        SELECT
            *
        FROM
            contracts
        WHERE
            start >= "2021-01-01"
    ''', conn)

df

In [ ]:
df["start"].min()

## Jak sortować wczytane rekordy?

In [ ]:
# sortowanie po jednej kolumnie
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('''
        SELECT
            name,
            age
        FROM
            employees
        ORDER BY name
    ''', conn)

df

In [ ]:
# sortowanie po dwóch kolumnach
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('''
        SELECT
            name,
            age
        FROM
            employees
        ORDER BY name, age
    ''', conn)

df

In [ ]:
# sortowanie malejące
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('''
        SELECT
            name,
            age
        FROM
            employees
        ORDER BY name DESC, age ASC
    ''', conn)

df

## Jak ograniczyć ilość wczytywany rekordów?

In [ ]:
# słowo kluczowe LIMIT - ograniczenie ilości wyników
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('''
        SELECT
            name,
            age
        FROM
            employees
        ORDER BY name
        LIMIT 10
    ''', conn)

df

In [ ]:
# słowo kluczowe LIMIT można łączyć z OFFSET
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('''
        SELECT
            name,
            age
        FROM
            employees
        ORDER BY name
        LIMIT 10 OFFSET 10
    ''', conn)

df

## Jak grupować dane przed ich zaciągnięciem?

In [ ]:
# instrukcja GROUP BY - grupowanie wyników
# instrukcja COUNT - zliczanie wyników
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('''
        SELECT
            department,
            COUNT(*) AS employees_count
        FROM
            employees
        GROUP BY department
    ''', conn)

df

In [ ]:
# inne przykładowe funkcje agregujące
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('''
        SELECT
            department,
            AVG(salary) AS avg_salary,
            MIN(salary) AS min_salary,
            MAX(salary) AS max_salary,
            SUM(salary) AS sum_salary
        FROM
            employees
        GROUP BY department
    ''', conn)

df

## Jak dodawać warunki w trakcie grupowania?

In [ ]:
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('''
        SELECT
            department,
            AVG(salary) AS avg_salary
        FROM
            employees
        -- WHERE musi być przed GROUP BY
        WHERE department = "HR" OR department = "IT"
        GROUP BY department
        HAVING avg_salary > 5000
    ''', conn)

df

## Jak znajdować unikalne wartości?

In [ ]:
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('''
        SELECT
            DISTINCT department
        FROM
            employees
    ''', conn)

df

## Jak znajdować unikatowe wiersze?

In [ ]:
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('''
        SELECT
            DISTINCT name, department
        FROM
            employees
        WHERE department IN ("HR", "IT")
        ORDER BY name
    ''', conn)

df

## Łączenie danych - usuwanie części nie pokrywających się

In [ ]:
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('''
        SELECT
            name,
            surname,
            department,
            contracts.type AS contract_type,
            contracts.start AS contract_start,
            contracts.end AS contract_end
        FROM
            employees
        JOIN contracts
        ON employees.id = contracts.employee_id
    ''', conn)

df

## Łączenie danych - ochraniamy z lewej

In [ ]:
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('''
        SELECT
            name,
            surname,
            department,
            contracts.type AS contract_type,
            contracts.start AS contract_start,
            contracts.end AS contract_end
        FROM
            employees
        LEFT JOIN contracts
        ON employees.id = contracts.employee_id
    ''', conn)

df

## Łączenie danych - ochraniamy z prawej

In [ ]:
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('''
        SELECT
            name,
            surname,
            department,
            contracts.type AS contract_type,
            contracts.start AS contract_start,
            contracts.end AS contract_end
        FROM
            employees
        RIGHT JOIN contracts
        ON employees.id = contracts.employee_id
    ''', conn)

df

## Łączenie danych - chronimy wszystkie tabele

In [ ]:
with sqlite3.connect('learning_select.db') as conn:
    df = pd.read_sql('''
        SELECT
            name,
            surname,
            department,
            contracts.type AS contract_type,
            contracts.start AS contract_start,
            contracts.end AS contract_end
        FROM
            employees
        FULL JOIN contracts
        ON employees.id = contracts.employee_id
    ''', conn)

df